In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import csv
import timeit

from os import listdir
from os.path import isfile, join
from IPython.display import clear_output

In [ ]:
data_dir = '/Users/yangjuehan/parse mmf/N-MFP2_csv/'
errorlog = pd.read_csv(data_dir + 'error_log.csv', dtype = str)
error_cik = [x for x in list(errorlog['cik2'].values)]
error_acc = [x for x in list(errorlog['accession_num'].values)]
error_paths = [x[0]+'_'+x[1]+'.csv' for x in zip(error_cik,error_acc)]

allpaths = pd.read_csv(data_dir + 'xmlpath.csv', dtype = str)
cik = [x for x in list(allpaths['cik2'].values)]
acc = [x for x in list(allpaths['accession_num'].values)]
filepaths = [x[0]+'_'+x[1]+'.csv' for x in zip(cik,acc)]

In [ ]:
crawledpaths = []
for x in filepaths:
    if x not in error_paths:
        crawledpaths.append(x)
len(error_paths), len(filepaths), len(crawledpaths)

In [ ]:
N = len(crawledpaths)
N_blocks = 10
block_len = int(N/N_blocks)
res_len = N%block_len
print(N,N_blocks,block_len,res_len)

In [ ]:
for i in range(N_blocks):
    if i < (N_blocks-1):
        block_paths = crawledpaths[i*block_len:(i+1)*block_len]
    else:
        block_paths = crawledpaths[i*block_len:]
    
    data_path = data_dir + 'NMFP2_data_' + str(i+1) + '.csv'
    
    start = timeit.default_timer()
    n = 0
    data = pd.DataFrame()
    for f in block_paths:
        n += 1
        df = pd.read_csv(data_dir + f)
        data = data.append(df, ignore_index = True, sort = False)

        if n%10 == 0:
            clear_output(wait = True)
            stop = timeit.default_timer()
            perc_run = n/block_len
            multiple_remain = (block_len-n)/n
            mins_elapse = (stop-start)/60
            
            print('Block: {}'.format(i+1))
            print('{:.2f}% finished'.format(perc_run*100))
            print('Current run time: {:.1f} minutes.'.format(mins_elapse))
            print('Expected remaining run time: {:.1f} minutes'.format(mins_elapse*multiple_remain))
    data.to_csv(data_path)

In [ ]:
final_data = pd.DataFrame()
for i in range(N_blocks):
    data_path = data_dir + 'NMFP2_data_' + str(i+1) + '.csv'
    final_data = final_data.append(pd.read_csv(data_path), ignore_index = True, sort = False)
final_data.to_csv('NMFP2_data.csv')